### Purpose

For the analysis we need to construct some phenotypic and covariant data. This notebook loads

* PCS's from the MRC IEU UK Biobank file
* Removal ID's, so we exclude those whom have removed consent
* A linker between our application of 41382 to the IEU common
* Data extracted from application 41382 on BMI, gender and YoB

Then combines the PC's from the IEU to your application, and converts our IID's to the IID type of the genetic file

In [1]:
from csvObject import CsvObject, write_csv
from miscSupports import terminal_time
import statsmodels.formula.api as smf
from pathlib import Path
import pandas as pd

common_path = r"Z:\UKB\GeographicID\Paper Data Extraction\Construction Files"
project_path = r"Z:\UKB\GeographicID\Paper Data Extraction\SB_Papers\SW_GWAS"

print(f"Set Environment {terminal_time()}")

Set Environment 15:59


In [2]:
# Load the principle components, set them as IID: [PC1, PC2, ... PC(N)]
pca_file = CsvObject(Path(common_path, "IEU_PCs_1_40.txt"), file_headers=False)
pc_dict = {row[0]: [r for r in row[2:] if r != ""] for row in pca_file.row_data}

# Load removal file for exclusion on IID
removal_ids = CsvObject(Path(common_path, "UKB_Withdrawal_IDs.csv"), set_columns=True)

# Set a linker between 41382: IEU Common
linker = {app: ieu for ieu, app in CsvObject(Path(common_path, "Linker.csv")).row_data}

# Load phenotype (BMI), gender, and year of birth from the data extraction from 41382
variables = CsvObject(Path(project_path, "Variables.csv"))

print(f"Loaded required files {terminal_time()}")

Loaded required files 9:59


In [32]:
analysis_rows = []
for iid, gender, yob, phenotype in variables.row_data:
    
    # If the iid is not set as withdrawn and is within the linker file between our applications
    if (iid not in removal_ids[0]) and (iid in linker.keys()) and (gender != "") and (yob != "") and (phenotype != ""):
        # Extract the PC via the linker and then append to the output container
        pcs = pc_dict[linker[iid]]
        analysis_rows.append([linker[iid], linker[iid], phenotype, gender, yob] + pcs)
        
print(f"Set IID file {terminal_time()}")

Set IID file 10:45


In [34]:
# Validate header length == row length
headers = ["FID", "IID", "BMI", "Gender", "YoB"] + [f"PC{i}" for i in range(1, 41)]
print(f"Header length is {len(headers)}: Row length is {len(analysis_rows[0])}")

Header length is 45: Row length is 45


In [35]:
# Write the file to disk
write_csv(project_path, "Analysis", headers, analysis_rows)
print(f"Output written at {terminal_time()}")

Output written at 10:55


### Create phenotype residuals

For several of our analysis runs we will need the residualised phenotype. This demonstrates the example of this dataset 
and writes it out for a comparision to the GWAS pipeline.

In [3]:
# Load analysis sample
analysis = pd.read_csv(Path(project_path, "Analysis.csv"))
print(f"Set analysis data-frame {terminal_time()}")

Set analysis data-frame 16:1


In [9]:
# Set the formula as phenotype ~ all other explanatory variables then run the OLS
formula = "BMI~" + "+".join([h for h in analysis.columns[3:]])
result = smf.ols(formula=formula, data=analysis).fit()
print(result.summary())
print(f"Set residualised phenotype {terminal_time()}")

                            OLS Regression Results                            
Dep. Variable:                    BMI   R-squared:                       0.017
Model:                            OLS   Adj. R-squared:                  0.017
Method:                 Least Squares   F-statistic:                     203.2
Date:                Mon, 17 May 2021   Prob (F-statistic):               0.00
Time:                        16:04:15   Log-Likelihood:            -1.4472e+06
No. Observations:              486262   AIC:                         2.895e+06
Df Residuals:                  486219   BIC:                         2.895e+06
Df Model:                          42                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept     84.1879      1.657     50.815      0.0

In [10]:
with_res = pd.concat([analysis, pd.DataFrame(result.resid, columns=["resBMI"])], axis=1)
print(with_res)

               FID         IID      BMI  Gender   YoB       PC1         PC2  \
0       IEU6498273  IEU6498273  35.6791       1  1956 -11.43620    4.216290   
1       IEU2766806  IEU2766806  38.4072       0  1943  -7.16004    2.184030   
2       IEU4946230  IEU4946230  24.8410       0  1947 -11.37910    0.741912   
3       IEU6907277  IEU6907277  26.3059       0  1940 -11.85220    3.739760   
4       IEU4159466  IEU4159466  24.5399       0  1955 -12.37480    1.926890   
...            ...         ...      ...     ...   ...       ...         ...   
486257  IEU3797450  IEU3797450  29.6208       1  1949 -13.76680    4.021320   
486258  IEU6382416  IEU6382416  22.7765       0  1960 -11.37570    2.297150   
486259  IEU5291816  IEU5291816  34.6746       1  1940 -13.35250    2.187330   
486260  IEU7325037  IEU7325037  27.6097       1  1946  90.58530 -136.197000   
486261  IEU6604467  IEU6604467  29.7250       1  1945 -13.15790    5.486830   

              PC3       PC4        PC5  ...      PC

In [12]:
with_res.to_csv(Path(project_path, "PhenoResiduals40PC.csv"))
print(f"Written residualised phenotypes {terminal_time()}")

Written residualised phenotypes 16:15
